# GenomeDK's ABC

Learn your way around the basics of the `GenomeDK` cluster.

## Infrastructure

`GenomeDK` is a **computing cluster**, i.e. a set of interconnected computers (nodes). `GenomeDK` has:

- **computing nodes** used for running programs (~15000 cores)
- **storage nodes** storing data in many hard drives (~23 PiB)
- a **network** making nodes talk to each other
- a **frontend** node from which you can send your programs to a node to be executed
- a **queueing system** called *slurm* to prioritize the users' program to be run

- **Logging into GenomeDK** happens through the command ^[both in Linux/Mac terminal and Windows Powershell. Powershell might require `ssh.exe` instead of `ssh`]

    ```{.bash}
    [local]$  ssh USERNAME@login.genome.au.dk
    ```

- When first logged in, **setup the 2-factor authentication** by 
    - showing a QR-code with the command

        ```{.bash}
        [fe-open-01]$  gdk-auth-show-qr
        ```
    - scanning it with your phone's Authenticator app ^[e.g. Microsoft Authenticator, Google Authenticator, ...].


## Access without password

It is nice to avoid writing the password at every access. If you are on the cluster, exit from it to go back to your local computer

```{.bash}
exit
```

&nbsp;

Now we generate an RSA key.

```{.bash}
ssh-keygen -t ed25519
```

Always press <kbd>Enter</kbd> and do not insert any password when asked. 

We create a folder on the cluster called `.ssh` to contain the RSA key we created

```{.bash}
ssh <USERNAME>@login.genome.au.dk mkdir -p .ssh
```
&nbsp;

and finally send the RSA public key to the cluster, into the file `authorized_keys`

```{.bash}
cat ~/.ssh/id_rsa.pub | ssh username@login.genome.au.dk 'cat >> .ssh/authorized_keys'
```
&nbsp;

this time will be the last where you are asked your password when logging in from your computer.

# File System (FS) on GenomeDK

Directory structure and how to navigate it

## How is the FS organized

:::: {.columns}

::: {.column width="50%" }

Folders and files follow a tree-like structure

- `/` is the root of the filesystem - nothing is above that
- `home` and `faststorage` are two **root folders**
- projects are in `faststorage` and **linked** to your home
- you can reach files and folders with a **path**

:::

::: {.column width="50%"}

![](img/complexTree.png){fig-align="center"}

:::

::::

---

:::: {.columns}

::: {.column width="50%" }


In [ ]:
#| echo: false
from jupyterquiz import display_quiz
git_url='https://raw.githubusercontent.com/hds-sandbox/scverse-2024-workshop/main/Questions/quiz00.json'
display_quiz(git_url, 1, shuffle_answers=True)

See the content of the current folder with 

```{.bash}
ls
```

or to see more details

```{.bash}
ls -lh
```

&nbsp;

:::{.callout-warning}
do not fill up your home with data. It has a **limited amount of storage** (a quota of 100GB).
:::

# Project management

&nbsp;

It is easy to start creating files everywhere in your project folders. Data, analysis files, results and the like. 

&nbsp;

**Managing your folders rationally** is the best way of finding your way around. Especially when getting back to your analysis after long time.

## Creation

You need a project from which you can run your programs. Request a project with the command

```{.bash}
gdk-project-request -g <project_name>
```

This creates a folder with the desired name. You should be able to go into that folder:

```{.bash}
cd <project_name>
```

&nbsp;

You can see how many resources your projects are using with 

```{.bash}
gdk-project-usage
```

## Folders management

It is important to

- have a coherent folder structure
- backup only important things (raw data, analysis scripts)

&nbsp;

Remember: Storage cost >> Computation cost

If your project has many users, a good structure can be

![](img/multistructure.png)

&nbsp; 

```{.bash}
mkdir -p Backup/Data Workspaces/username1 Workspaces/username2
ln -s Backup/Data/ Data
```

`wget` has many options you can use, but what shown in the example above is what you need most times. You can see them with the command

```{.bash}
wget --help
```

&nbsp;

Also, you can find [this cheatsheet](https://opensource.com/sites/default/files/gated-content/osdc_cheatsheet-wget-2021.10.21.pdf) useful for remembering the commands to most of the things you can think about downloading files using wget. [At this page](https://opensource.com/article/21/10/linux-wget-command) there are also some concrete examples for `wget`.

## SCP transfer

`SCP` (Secure Copy Protocol) can transfer files securely

- between a LOCAL and a REMOTE host
- between TWO REMOTE hosts

&nbsp;

You can use it to transfer files **from your pc to GenomeDK and viceversa**, but also **between GenomeDK and another computing cluster** (for example, downloading data from a collaborator, which resides on a different remote computing system).

If you want to copy an entire folder, use the option -r (recursive copy). The previous examples become

```{.bash}
scp -r /home/my_laptop/Documents/folder \
       username@login.genome.au.dk:/home/username/my_project/
```

&nbsp;

and

```{.bash}
scp -r username@login.genome.au.dk:/home/username/my_project/folder \
       /home/my_laptop/Documents/
```

&nbsp;

A few more options are available and you can see them with the command ``` scp --help ```.

## Rsync transfer

Differently from `scp`, you can use `rsync` to **syncronize files and folders between two locations**. It copies only the changes in the data and not all of it every time.

&nbsp;

Copying a file or a folder between your computer and `GenomeDK` works exactly as in `scp`. For example

```{.bash}
rsync --progress -r \
      username@login.genome.au.dk:/home/username/my_project/folder \
      /home/my_laptop/Documents/
```

where we add an option to show a progress bar

Press on `Quick Connect`. As a result, you will establish a secure connection to `GenomeDK`. On the left-side browser you can see your local folders and files. On the right-side, the folders and files on `GenomeDK` starting from your `home`.\

![](./img/filezilla1.png){fig-align="center"}

The **download** process works similarly using the right-side browser and choosing the destination folder on the left-side browser.

![](./img/filezilla3.png){fig-align="center"}

![](./img/virtualenvs.png)

## Conda

&nbsp;

Conda is **both** a virtual environment and a package manager.

- easy to use and understand
- can handle quite big environments
- environments are easily shareable
- a [large archive](https://anaconda.org) (Anaconda) of packages
- active community of people archiving their packages on Anaconda

## Installation

&nbsp;

Just download and execute the installer by
```{.bash}
wget https://github.com/conda-forge/miniforge/releases/latest/download/Miniforge3-Linux-x86_64.sh -O miniforge.sh
chmod +x miniforge.sh
bash miniforge.sh -b
./miniforge3/bin/conda init bash
```

After a few `ENTER`s and `YES`'s you should get the installation done. Run

```{.bash}
source ~/.bashrc
```

and doublecheck that `conda` works:

```{.bash}
conda info
```


Look at the settings in your conda installation. They are saved in the file `~/.condarc`

```{.bash}
cat ~/.condarc
```

## Create an environment

&nbsp;

An empty environment called `test_1`:

```{.bash}
conda create -n test_1
```

&nbsp;

You can list all the environments available:

```{.bash}
conda env list
```
```
> # conda environments:
> #
> base      *  /home/samuele/miniconda3
> test_1       /home/samuele/miniconda3/envs/test_1
```

## Activate and deactivate

&nbsp;

To use an environment

```{.bash}
conda activate test_1
```

Deactivation happens by

```{.bash}
conda deactivate
```

## Manage an environment

### Package installation

Conda puts together the dependency trees of requested packages to find all compatible dependencies versions.

![Figure: A package's dependency tree with required versions on the edges](img/condatree.png)

![Figure: suggested commands to install the package](img/anaconda2.png){width=10cm}

:::{.callout-note title="Repositories"}

packages are archived in repositories. Typical ones are `bioconda`, `conda-forge`, `r`, `anaconda`.

`conda-forge` packges are often more up-to-date, but a few times show compatibility problems with other packages.
:::

### Installation from a list of packages

You can export all the packages you have installed over time in your environment:

```{.bash}
conda env export --from-history > environment.yml
```

which looks like 

```{.yaml}
name: test_1
channels:
 - bioconda
 - conda-forge
 - defaults
 - r
dependencies:
 - bioconda::bioconductor-deseq2
 - conda-forge::r-tidyr
```

You can use the `yml` file to create an environment:

&nbsp;

```
conda env create -p test_1_from_file -f ./environment.yml
```

&nbsp;

Environment files are very useful when you want to **share environments with others**, especially when the package list is long.

## Useful links for conda:

- [Conda cheat sheet](https://docs.conda.io/projects/conda/en/4.6.0/_downloads/52a95608c49671267e40c689e0bc00ca/conda-cheatsheet.pdf) with all the things you can do to manage environments

- [Anaconda](http://anaconda.org) where you can search for packages

# Running a Job

&nbsp;

Running programs on a computing cluster happens through **jobs**. 

&nbsp;

Learn how to get hold of **computing resources** to run your programs.

## What is a job on a HPC

A computational task **executed on requested HPC resources** (computing nodes), which are handled by the **queueing system** (SLURM).

![](img/Job-on-cluster.png){fig-align="center"}

Front-end nodes are limited in memory and power, and **should only be for basic operations** such as

- starting a new project

- small folders and files management

- small software installations

and in general you **should not** use them to run computations. This might slow down other users.

To run an interactive job simply use the command

```
[fe-open-01]$ srun --mem=<GB_of_RAM>g -c <nr_cores> --time=<days-hrs:mins:secs>  --account=<project_name> --pty /bin/bash
```

For example

```
[fe-open-01]$ srun --mem=32g -c 2 --time=6:0:0  --account=<project_name> --pty /bin/bash
```

The **queueing system** makes you wait based on the resources you ask and how busy the nodes are. When you get **assigned a node**, the resources are available. The node name is **shown in the prompt**.

```
[<username>@s21n32 ~]$
```

## Batch script (sbatch)

Useful to **run a program non-interactively**, usually for longer time than a short interaction. A batch script contains 

- the desired **resources**
- the sequence of **commands** to be executed

and

- has a filename **without spaces** (forget spaces from now on)
- starts with `#!/bin/bash` to know in which language ('bash') the commands are written into

Send the script to the queueing system:

```{.bash}
sbatch align.sh
```
```
Submitted batch job 33735298
```

&nbsp;

Interrogate SLURM about the specific job

```{.bash}
jobinfo 33735298
```

```
>Name                : align.sh
>User                : samuele
>Account             : my_project
>Partition           : short
>Nodes               : s21n43
>Cores               : 8
>GPUs                : 0
>State               : RUNNING
>...
```

To observe in real time the latest output of the command in the job, you can refresh the last lines of the log file for the specific job:

```{.bash}
watch tail align.sh-33735928.out
```

&nbsp;

To look at the whole file, you can run at any time

```{.bash}
less -S align.sh-33735928.out
```

This can be useful for debugging, when for example a command gives an error and the job interrupts.



## Other ways of running jobs

Beyond `sbatch`, you can use other tools for workflows which are

- **modular and composable**: sequences of commands can be applied in various contexts, composed together in the desired ordering
- **scalable and parallel** handling many sequences of operations parallelly or interdependently
- **flexible** where repetitive operations can be automatized over multiple applications

---

Some workflow tools:

- [Snakemake](https://snakemake.readthedocs.io/en/stable/)
- [NextFlow](https://www.nextflow.io/)
- [Gwf](https://gwf.app/)

&nbsp;

`Gwf` has an easy `python` syntax instead of its own language to write workflows.

&nbsp;

You need to know some basic `python` to use `Gwf`, but it is worth the effort.